In [1]:
# mount to my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Tue Feb 24 17:15:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
import torch
print(torch.cuda.is_available()) # should be true
print("GPU:", torch.cuda.get_device_name(0)) # should be NVIDIA A100-SXM4-40GB

True
GPU: NVIDIA A100-SXM4-40GB


In [44]:
# run if you need to reclone
!rm -r /content/tutorial_data_synthesis_and_evaluation

In [4]:
%cd /content
!git clone https://github.com/sfurlow8/tutorial_data_synthesis_and_evaluation.git
%cd tutorial_data_synthesis_and_evaluation

/content
Cloning into 'tutorial_data_synthesis_and_evaluation'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 218 (delta 53), reused 66 (delta 20), pack-reused 97 (from 1)
Receiving objects: 100% (218/218), 31.03 MiB | 15.97 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/tutorial_data_synthesis_and_evaluation


In [5]:
!ls /content/drive/MyDrive/mimic_synthetic_data/Data/preprocessing/test

column_list_only_diagnosis.npy
mapping_tables
min_max_log.npy
normalized_testing_data.csv
normalized_training_data.csv
one_y_outcome.npy
original_testing_data.csv
original_training_data.csv
patient_bmi_bp_dict.npy
patient_concept_mat_df.csv
patient_concept_mat_only_diagnosis.npy
patient_diagnosis_dict.npy
patient_ICD_diagnosis_final_dict.npy
patient_phecode_diagnosis_final_dict.npy
patient_prescriptions_dict.npy
patient_prescriptions_ingr_rxcui_final_dict.npy
patient_prescriptions_rxcui_final_dict.npy
patient_procedures_hcpcs_dict.npy
patient_procedures_icd_dict.npy
patients_anchor_age_not_last_v_y.npy
patients_dict_w_admission_n_die_inhosp.npy
patients_dict_w_admission.npy
patients_final_dict.npy
patients_w_icd_diag_9_10.npy
patients_w_icd_proc_9_10.npy
preprocessed_training_data.csv
rxnav_cache.json


In [14]:
!pip install -r requirements_colab.txt

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.6 MB/s eta 0:00:00
  Using cached matplotlib-3.9.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached shap-0.45.1-cp312-cp312-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.0 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.2 M

In [25]:
!pip uninstall tensorflow -y

In [26]:
# Install TensorFlow 2.16.2 (earliest stable version supporting Python 3.12)
# Install compatible protobuf version first to avoid symbol mismatch errors
!pip install "protobuf==4.25.3" --force-reinstall
!pip install "tensorflow==2.16.2"
!pip install "protobuf==4.25.3" --force-reinstall --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.5
    Uninstalling protobuf-6.33.5:
      Successfully uninstalled protobuf-6.33.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.8/590.8 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.7 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.4
    Uninstalling ml_dtypes-0.5.4:
      Successfully uninstalled ml_dtypes-0.5.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.20.0
    Uninstalling tensorboard-2.20.0:
      Successfully uninstalled tensorboard-2.20.0


  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3


In [7]:
!python GAN_training.py --gpu_id 0 --model_id test2

2026-02-24 17:16:12.773583: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-24 17:16:12.789672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771953372.809332    1491 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771953372.815757    1491 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771953372.830925    1491 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 